# Machine Learning with PyTorch and Scikit-Learn  

#### CHANGES ####
Moved imports to a dedicated cell.
Moved magic nubmers to a dedicated constants cell

Imports

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
import numpy as np
import numpy as np


Constants

In [ ]:
# THESE ARE CONSTANTS
RANDOM_SEED: int = 42

## Obtaining and preparing the MNIST dataset

The MNIST dataset is publicly available at http://yann.lecun.com/exdb/mnist/ and consists of the following four parts:

- Training set images: train-images-idx3-ubyte.gz (9.9 MB, 47 MB unzipped, 60,000 examples)
- Training set labels: train-labels-idx1-ubyte.gz (29 KB, 60 KB unzipped, 60,000 labels)
- Test set images: t10k-images-idx3-ubyte.gz (1.6 MB, 7.8 MB, 10,000 examples)
- Test set labels: t10k-labels-idx1-ubyte.gz (5 KB, 10 KB unzipped, 10,000 labels)



In [ ]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X.values
y = y.astype(int).values

print(X.shape)
print(y.shape)

Normalize to [-1, 1] range:

In [ ]:
X = ((X / 255.) - .5) * 2

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=10000, random_state=123, stratify=y)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_temp, y_temp, test_size=5000, random_state=123, stratify=y_temp)


# optional to free up some memory by deleting non-used arrays:
del X_temp, y_temp, X, y

## Implementing a multi-layer perceptron

Here, for part 1, An additional layer was added.

In [ ]:
##########################
### MODEL
##########################

def sigmoid(z):                                        
    return 1. / (1. + np.exp(-z))


def int_to_onehot(y, num_labels):

    ary = np.zeros((y.shape[0], num_labels))
    for index, val in enumerate(y):
        ary[index, val] = 1

    return ary

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    z_deriv = np.array(z, copy=True)
    z_deriv[z_deriv <= 0] = 0
    z_deriv[z_deriv > 0] = 1
    return z_deriv.astype(float)

class NeuralNetMLP_2_HiddenLayers:
    def __init__(self, 
                 num_features: int, 
                 hidden_layer1_size: int, 
                 hidden_layer2_size: int,
                 num_classes: int, 
                 random_seed=RANDOM_SEED
                 ):
        super().__init__()
        
        self.num_classes = num_classes
        
        # hidden
        rng = np.random.RandomState(random_seed)
        
        self.weight_hidden_layer_1 = rng.normal(
            loc=0.0, scale=0.1, size=(hidden_layer1_size, num_features))
        self.bias_h1 = np.zeros(hidden_layer1_size)

        self.weight_hidden_layer_2 = rng.normal(
            loc=0.0, scale=0.1, size=(hidden_layer2_size, hidden_layer1_size))
        self.bias_h2 = np.zeros(hidden_layer2_size)

        # output
        self.weight_out = rng.normal(
            loc=0.0, scale=0.1, size=(num_classes, hidden_layer2_size))
        self.bias_out = np.zeros(num_classes)
        
    def forward(self, x):
        # Hidden layer 1
        # input dim: [n_examples, n_features] dot [n_hidden1, n_features].T
        z_hidden_layer_1 = np.dot(x, self.weight_hidden_layer_1.T) + self.bias_h1
        a_hidden_layer_1 = sigmoid(z_hidden_layer_1)

        # Hidden layer 2
        # input dim: [n_examples, n_hidden1] dot [n_hidden2, n_hidden1].T
        z_hidden_layer_2 = np.dot(a_hidden_layer_1, self.weight_hidden_layer_2.T) + self.bias_h2
        a_hidden_layer_2 = sigmoid(z_hidden_layer_2)

        # Output layer
        # input dim: [n_examples, n_hidden2] dot [n_classes, n_hidden2].T
        # output dim: [n_examples, n_classes]
        z_out = np.dot(a_hidden_layer_2, self.weight_out.T) + self.bias_out
        a_out = sigmoid(z_out)
        return a_hidden_layer_1, a_hidden_layer_2, a_out

    def backward(self, x, a_hidden_layer_1, a_hidden_layer_2, a_out, y):

        # onehot encoding
        y_onehot = int_to_onehot(y, self.num_classes)

        #########################
        # Output layer gradients
        #########################
        d_loss__d_a_out = 2.0 * (a_out - y_onehot) / y.shape[0]
        d_a_out__d_z_out = a_out * (1.0 - a_out)             # sigmoid'
        delta_out = d_loss__d_a_out * d_a_out__d_z_out       # (N, C)

        # W_out: (C, H2), b_out: (C,)
        d_loss__dw_out = np.dot(delta_out.T, a_hidden_layer_2)   # (C, H2)
        d_loss__db_out = np.sum(delta_out, axis=0)               # (C,)

        ########################################
        # Hidden layer 2 gradients (sigmoid)
        ########################################
        # backprop into a_hidden_layer_2: (N, H2)
        d_loss__d_a2 = np.dot(delta_out, self.weight_out)        # (N, H2)
        d_a2__d_z2 = a_hidden_layer_2 * (1.0 - a_hidden_layer_2) # sigmoid'
        delta_2 = d_loss__d_a2 * d_a2__d_z2                      # (N, H2)

        # W2: (H2, H1), b2: (H2,)
        d_loss__dw_h2 = np.dot(delta_2.T, a_hidden_layer_1)      # (H2, H1)
        d_loss__db_h2 = np.sum(delta_2, axis=0)                  # (H2,)

        ########################################
        # Hidden layer 1 gradients (sigmoid)
        ########################################
        # backprop into a_hidden_layer_1: (N, H1)
        d_loss__d_a1 = np.dot(delta_2, self.weight_hidden_layer_2)   # (N, H1)
        d_a1__d_z1 = a_hidden_layer_1 * (1.0 - a_hidden_layer_1)     # sigmoid'
        delta_1 = d_loss__d_a1 * d_a1__d_z1                          # (N, H1)

        # W1: (H1, F), b1: (H1,)
        d_loss__dw_h1 = np.dot(delta_1.T, x)                      # (H1, F)
        d_loss__db_h1 = np.sum(delta_1, axis=0)                   # (H1,)

        return (d_loss__dw_out, d_loss__db_out,
                d_loss__dw_h2,  d_loss__db_h2,
                d_loss__dw_h1,  d_loss__db_h1)


In [ ]:
model = NeuralNetMLP_2_HiddenLayers(num_features=28*28,
                     hidden_layer1_size=50,
                     hidden_layer2_size=50,
                     num_classes=10)

## Coding the neural network training loop

Defining data loaders:

In [ ]:
num_epochs = 50
minibatch_size = 100


def minibatch_generator(X, y, minibatch_size):
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    for start_idx in range(0, indices.shape[0] - minibatch_size 
                           + 1, minibatch_size):
        batch_idx = indices[start_idx:start_idx + minibatch_size]
        
        yield X[batch_idx], y[batch_idx]

        
# iterate over training epochs
for i in range(num_epochs):

    # iterate over minibatches
    minibatch_gen = minibatch_generator(
        X_train, y_train, minibatch_size)
    
    for X_train_mini, y_train_mini in minibatch_gen:

        break
        
    break
    
print(X_train_mini.shape)
print(y_train_mini.shape)

Defining a function to compute the loss and accuracy

In [ ]:
def mse_loss(targets, probas, num_labels=10):
    onehot_targets = int_to_onehot(targets, num_labels=num_labels)
    return np.mean((onehot_targets - probas)**2)


def accuracy(targets, predicted_labels):
    return np.mean(predicted_labels == targets) 


_, _, probas = model.forward(X_valid)
mse = mse_loss(y_valid, probas)

predicted_labels = np.argmax(probas, axis=1)
acc = accuracy(y_valid, predicted_labels)

print(f'Initial validation MSE: {mse:.1f}')
print(f'Initial validation accuracy: {acc*100:.1f}%')

In [ ]:
def compute_mse_and_acc(nnet, X, y, num_labels=10, minibatch_size=100):
    mse, correct_pred, num_examples = 0., 0, 0
    minibatch_gen = minibatch_generator(X, y, minibatch_size)
        
    for i, (features, targets) in enumerate(minibatch_gen):

        _, _, probas = nnet.forward(features)
        predicted_labels = np.argmax(probas, axis=1)
        
        onehot_targets = int_to_onehot(targets, num_labels=num_labels)
        loss = np.mean((onehot_targets - probas)**2)
        correct_pred += (predicted_labels == targets).sum()
        
        num_examples += targets.shape[0]
        mse += loss

    mse = mse/(i+1)
    acc = correct_pred/num_examples
    return mse, acc

In [ ]:
mse, acc = compute_mse_and_acc(model, X_valid, y_valid)
print(f'Initial valid MSE: {mse:.1f}')
print(f'Initial valid accuracy: {acc*100:.1f}%')

In [ ]:
def train(model, X_train, y_train, X_valid, y_valid, num_epochs,
          learning_rate=0.1):

    epoch_loss = []
    epoch_train_acc = []
    epoch_valid_acc = []

    for e in range(num_epochs):

        minibatch_gen = minibatch_generator(X_train, y_train, minibatch_size)

        for X_train_mini, y_train_mini in minibatch_gen:

            #### Forward ####
            a1, a2, a_out = model.forward(X_train_mini)

            #### Backward ####
            (dW_out, db_out,
             dW2, db2,
             dW1, db1) = model.backward(
                X_train_mini,
                a1, a2,
                a_out,
                y_train_mini
            )

            #### Update ####
            model.weight_hidden_layer_1 -= learning_rate * dW1
            model.bias_h1              -= learning_rate * db1

            model.weight_hidden_layer_2 -= learning_rate * dW2
            model.bias_h2              -= learning_rate * db2

            model.weight_out -= learning_rate * dW_out
            model.bias_out   -= learning_rate * db_out

        #### Epoch Logging ####
        train_mse, train_acc = compute_mse_and_acc(model, X_train, y_train)
        valid_mse, valid_acc = compute_mse_and_acc(model, X_valid, y_valid)

        train_acc *= 100.0
        valid_acc *= 100.0

        epoch_train_acc.append(train_acc)
        epoch_valid_acc.append(valid_acc)
        epoch_loss.append(train_mse)

        print(f'Epoch: {e+1:03d}/{num_epochs:03d} '
              f'| Train MSE: {train_mse:.2f} '
              f'| Train Acc: {train_acc:.2f}% '
              f'| Valid Acc: {valid_acc:.2f}%')

    return epoch_loss, epoch_train_acc, epoch_valid_acc


In [ ]:
np.random.seed(RANDOM_SEED) # for the training set shuffling

epoch_loss, epoch_train_acc, epoch_valid_acc = train(
    model, X_train, y_train, X_valid, y_valid,
    num_epochs=50, learning_rate=0.1)

## Evaluating the neural network performance

In [ ]:
plt.plot(range(len(epoch_loss)), epoch_loss)
plt.ylabel('Mean squared error')
plt.xlabel('Epoch')
#plt.savefig('figures/11_07.png', dpi=300)
plt.show()

In [ ]:
plt.plot(range(len(epoch_train_acc)), epoch_train_acc,
         label='Training')
plt.plot(range(len(epoch_valid_acc)), epoch_valid_acc,
         label='Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(loc='lower right')
#plt.savefig('figures/11_08.png', dpi=300)
plt.show()

In [ ]:
test_mse, test_acc = compute_mse_and_acc(model, X_test, y_test)
print(f'Test accuracy: {test_acc*100:.2f}%')

Plot failure cases:

In [ ]:
X_test_subset = X_test[:1000, :]
y_test_subset = y_test[:1000]

_, _, probas = model.forward(X_test_subset)
test_pred = np.argmax(probas, axis=1)

misclassified_images = X_test_subset[y_test_subset != test_pred][:25]
misclassified_labels = test_pred[y_test_subset != test_pred][:25]
correct_labels = y_test_subset[y_test_subset != test_pred][:25]

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=5, 
                       sharex=True, sharey=True, figsize=(8, 8))
ax = ax.flatten()
for i in range(25):
    img = misclassified_images[i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
    ax[i].set_title(f'{i+1}) '
                    f'True: {correct_labels[i]}\n'
                    f' Predicted: {misclassified_labels[i]}')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
#plt.savefig('figures/11_09.png', dpi=300)
plt.show()